In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from selenium import webdriver
import urllib.request
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from time import sleep
import random
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException
from tqdm import tqdm

In [2]:
main_link = 'https://www.lazada.vn/catalog/?q=laptop'
path = 'chromedriver.exe'

# # Customize chrome displayLa
chrome_options = Options()
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-notifications')
chrome_options.add_argument('--disable-infobars')

service = Service(executable_path=path)
driver = webdriver.Chrome(service=service, options=chrome_options)

driver.get(main_link)

In [3]:
#Crawl the information displayed on the outside of the page
def get_Data_On_Page():
    #The information is definitely there
    Type = [elem.get_attribute("value") for elem in driver.find_elements(By.CSS_SELECTOR, ".search-box__input--O34g")]
    Title = [elem.text for elem in driver.find_elements(By.CSS_SELECTOR, ".RfADt [href]")]
    Link = [elem.get_attribute('href') for elem in driver.find_elements(By.CSS_SELECTOR, ".RfADt [href]")]
    Price_sale = [elem.text for elem in driver.find_elements(By.CSS_SELECTOR, ".ooOxS")]
    Sale_off = [elem.text for elem in driver.find_elements(By.CSS_SELECTOR , ".WNoq3")]

    
    #The information may or may not be available
    #If not, assign None
    Total_sold, Preview = [], []
    for i in range(1, len(Title)+1):
        try:
            one_total_sold = driver.find_element("xpath", "/html/body/div[3]/div/div[2]/div[1]/div/div[1]/div[2]/div[{}]/div/div/div[2]/div[5]/span[1]/span[1]".format(i))
            Total_sold.append(one_total_sold.text)
        except NoSuchElementException:
            Total_sold.append(None)

        try:
            one_preview = driver.find_element("xpath", "/html/body/div[3]/div/div[2]/div[1]/div/div[1]/div[2]/div[{}]/div/div/div[2]/div[5]/div/span".format(i))
            Preview.append(one_preview.text)
        except NoSuchElementException:
            Preview.append(None)
    
    Location = [elem.text for elem in driver.find_elements(By.CSS_SELECTOR, ".oa6ri")]
    if len(Type) == 1:
        Type = Type * len(Title)

    #Create a dictionary with the lists
    data = {
        "Type": Type,
        "Title": Title,
        "Link": Link,
        "Price_sale": Price_sale,
        "Sale_off": Sale_off,
        "Total_sold": Total_sold,
        "Preview": Preview,
        "Location": Location
    }

    # Convert the dictionary to a DataFrame
    df = pd.DataFrame(data)
    return df

In [4]:
#Call the function and get the DF
df_Laptop = get_Data_On_Page()
df_Laptop.index = np.arange(1, len(df_Laptop)+1)
print(df_Laptop)

      Type                                              Title  \
1   laptop  [6.6 SALE TO] MacBook Air 2020 13.3 inches M1-...   
2   laptop  Lenovo mới tinh core Intel i7 6560U laptop 15,...   
3   laptop  Máy Tính Xách Tay Bàn Phím Bao Gồm, Màng Bảo V...   
4   laptop  ASUS máy tính xách tay mới tinh Intel core i7 ...   
5   laptop  LaptopLENOVO THINKPAD X1 Carbon Gen 7 X1 Carbo...   
6   laptop  Laptop Dell Inspiron 7567 máy tính giá rẻ(Core...   
7   laptop  Lenovo Thinkpad / Tosiba / Dell Core i5 / RAM ...   
8   laptop  【Vietnam spot】【ASUS Factory】Monsprin 2024 new ...   
9   laptop  Mới[Mới 100%]【Lennovo Factory】Monsprin 2024 ne...   
10  laptop  Laptop cũ nhiều lựa chọn Pentium đến i3 i5 i7 ...   
11  laptop  Thanh lý Laptop Toshiba, core I5, ram 4Gb, ô c...   
12  laptop  DELL 7470 i7 16G 512G MÀN HÌNH 14 in CẢM ỨNG C...   
13  laptop  Laptop game và đồ họa giá tốt HP 6470b Core i5...   
14  laptop  Laptop Dell Latitude E7250 màn 12.5 - Core i7 ...   
15  laptop  Laptop Dell L

In [5]:
#Get the link and access each product to get more features
def get_Data_OneProduct_Link(link):
    driver.get(link)                            #Get link
    sleep(random.uniform(3, 5))                 #Random sleep to mimic human behavior and avoid getting blocked
    
    Price_original = [elem.text for elem in driver.find_elements(By.CSS_SELECTOR, ".notranslate.pdp-price.pdp-price_type_deleted.pdp-price_color_lightgray.pdp-price_size_xs")]
    Ship_price = [elem.text for elem in driver.find_elements(By.CSS_SELECTOR, ".delivery-option-item.delivery-option-item_type_standard .delivery-option-item__body .delivery-option-item__shipping-fee.no-subtitle")]
    Return = [elem.text for elem in driver.find_elements(By.CSS_SELECTOR, ".delivery-option-item.delivery-option-item_type_returnPolicy30 .delivery-option-item__body .delivery-option-item__info .delivery-option-item__title")]

    # Sale_rating & Ship_on_time & Chat_respone
    percentage = [elem.text for elem in driver.find_elements(By.CSS_SELECTOR, ".seller-info-value ")]
    Sale_rating = percentage[0] if len(percentage) > 0 else None
    Ship_on_time = percentage[1] if len(percentage) > 1 else None
    Chat_response = percentage[2] if len(percentage) > 2 else None


    #Cuộn trang từ từ để tải phần đánh giá sao
    while True:
        driver.execute_script("window.scrollBy(0, 400);")
        sleep(1)  #Đợi một chút để nội dung tải
        try:
            stars = driver.find_elements(By.CSS_SELECTOR, ".detail .percent")
            if len(stars) >= 5:
                break
        except NoSuchElementException:
            continue
    
    One_star, Two_star, Three_star, Four_star, Five_star = [], [], [], [], []
    stars = [elem.text for elem in stars]

    One_star.append(stars[4] if len(stars) > 4 else 'N/A')
    Two_star.append(stars[3] if len(stars) > 3 else 'N/A')
    Three_star.append(stars[2] if len(stars) > 2 else 'N/A')
    Four_star.append(stars[1] if len(stars) > 1 else 'N/A')
    Five_star.append(stars[0] if len(stars) > 0 else 'N/A')


    #Create a dictionary with the lists
    One_Product = {
        "Price_original": Price_original[0] if Price_original else None,
        "Ship_price": Ship_price[0] if Ship_price else None,
        "Return": Return[0] if Return else None,
        "Sale_rating": Sale_rating,
        "Ship_on_time": Ship_on_time,
        "Chat_response": Chat_response,
        "One_star": One_star,
        "Two_star": Two_star,
        "Three_star": Three_star,
        "Four_star": Four_star,
        "Five_star": Five_star
    }
    
    return One_Product 

In [6]:
#Initialize dictionary to store details for all products
Products = {
    "Price_original": [],
    "Ship_price": [],
    "Return": [],
    "Sale_rating": [],
    "Ship_on_time": [],
    "Chat_response": [],
    "One_star": [],
    "Two_star": [],
    "Three_star": [],
    "Four_star": [],
    "Five_star": []
}

#Create a progress bar to track the crawl process
for link in tqdm(df_Laptop.Link, desc="Crawling link product"):
    LinkProduct = get_Data_OneProduct_Link(link)
    for key in Products.keys():
        Products[key].append(LinkProduct[key])

Crawling link product: 100%|██████████| 40/40 [19:26<00:00, 29.16s/it]


In [7]:
#Add columns to dienthoai
for key, value in Products.items():
    df_Laptop[key] = value

In [8]:
print(df_Laptop)

      Type                                              Title  \
1   laptop  [6.6 SALE TO] MacBook Air 2020 13.3 inches M1-...   
2   laptop  Lenovo mới tinh core Intel i7 6560U laptop 15,...   
3   laptop  Máy Tính Xách Tay Bàn Phím Bao Gồm, Màng Bảo V...   
4   laptop  ASUS máy tính xách tay mới tinh Intel core i7 ...   
5   laptop  LaptopLENOVO THINKPAD X1 Carbon Gen 7 X1 Carbo...   
6   laptop  Laptop Dell Inspiron 7567 máy tính giá rẻ(Core...   
7   laptop  Lenovo Thinkpad / Tosiba / Dell Core i5 / RAM ...   
8   laptop  【Vietnam spot】【ASUS Factory】Monsprin 2024 new ...   
9   laptop  Mới[Mới 100%]【Lennovo Factory】Monsprin 2024 ne...   
10  laptop  Laptop cũ nhiều lựa chọn Pentium đến i3 i5 i7 ...   
11  laptop  Thanh lý Laptop Toshiba, core I5, ram 4Gb, ô c...   
12  laptop  DELL 7470 i7 16G 512G MÀN HÌNH 14 in CẢM ỨNG C...   
13  laptop  Laptop game và đồ họa giá tốt HP 6470b Core i5...   
14  laptop  Laptop Dell Latitude E7250 màn 12.5 - Core i7 ...   
15  laptop  Laptop Dell L

In [ ]:
#Convert to csv
Data_laptop_Page01 = df_Laptop.to_csv("Data_laptop_Page01.csv", index=False)